In [1]:
import pandas as pd
import numpy as np

In [2]:
promo = pd.read_csv("../data/promoted_content.csv.gz", index_col=False)
train = pd.read_csv("../data/clicks_train.csv.gz", index_col = False)
test  = pd.read_csv("../data/clicks_test.csv.gz", index_col = False)

In [14]:
train_ad = train[["ad_id"]].drop_duplicates()
test_ad = test[["ad_id"]].drop_duplicates()

In [9]:
test['clicked'] = -1
test.head()

,display_id,ad_id,clicked
0,16874594,66758,-1
1,16874594,150083,-1
2,16874594,162754,-1
3,16874594,170392,-1
4,16874594,172888,-1


In [11]:
total = pd.concat([train, test])
total.head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [16]:
total_ad = pd.concat([train_ad, test_ad])


In [17]:
total.to_csv("../interm/train_test_all.csv.gz", 
                index=False, compression="gzip")

In [18]:
selected_promo = total_ad.merge(promo, how = "left", on = "ad_id")

In [19]:
selected_promo.rename(columns = {"document_id"   : "ad_prom:document_id",
                                 "campaign_id"   : "ad_prom:campaign_id",
                                 "advertiser_id" : "ad_prom:advertiser_id",
                                }, 
                      inplace = True)

In [20]:
selected_promo.to_csv("../interm/selected_promo_content.csv.gz",
                     index=False, compression="gzip")

## selected promoted content joins with events
- join train with events on display_id give f1_train : (display, ad, doc_id, uuid, timestamp platform, geo_location)
- join f1_train with selected_promo on ad_id and doc_id give f2_train : (display, ad, ad:campagin_id, ad:advertiser_id, doc_id, timestamp platform, geo_location)

In [5]:
events = pd.read_csv("../data/events.csv.gz", index_col=False)

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
events["platform"].unique()

array([3, 2, 1, '2', '1', '3', '\\N'], dtype=object)

In [3]:
def toInt(s):
    if s != '\\N':
        s = int(s)
    else :
        s = -1
    return s

In [6]:
events["platform"] = events["platform"].map(toInt)

In [15]:
events.to_csv("../data/events_platformTypeCleaned.csv.gz",
             index=False, compression="gzip")

In [21]:
f1_all = total.merge(events, how = "left", on = "display_id")

In [22]:
f1_all.rename(columns={'document_id': 'events:document_id', 
                         'timestamp' : 'events:timestamp',
                         'platform': 'events:platform', 
                         'geo_location': 'events:geo_location',
                        }, 
                inplace=True)

In [23]:
f1_all.head()

,display_id,ad_id,clicked,uuid,events:document_id,events:timestamp,events:platform,events:geo_location
0,1,42337,0,cb8c55702adb93,379743,61,3,US>SC>519
1,1,139684,0,cb8c55702adb93,379743,61,3,US>SC>519
2,1,144739,1,cb8c55702adb93,379743,61,3,US>SC>519
3,1,156824,0,cb8c55702adb93,379743,61,3,US>SC>519
4,1,279295,0,cb8c55702adb93,379743,61,3,US>SC>519


In [24]:
f1_all.to_csv("../interm/f1_all.csv.gz",
                index=False, compression="gzip")

In [25]:
f2_all = f1_all.merge(selected_promo, how = 'left', on = 'ad_id')

In [26]:
f2_all.head()

,display_id,ad_id,clicked,uuid,events:document_id,events:timestamp,events:platform,events:geo_location,ad_prom:document_id,ad_prom:campaign_id,ad_prom:advertiser_id
0,1,42337,0,cb8c55702adb93,379743,61,3,US>SC>519,938164,5969,1499
1,1,42337,0,cb8c55702adb93,379743,61,3,US>SC>519,938164,5969,1499
2,1,139684,0,cb8c55702adb93,379743,61,3,US>SC>519,1085937,17527,2563
3,1,139684,0,cb8c55702adb93,379743,61,3,US>SC>519,1085937,17527,2563
4,1,144739,1,cb8c55702adb93,379743,61,3,US>SC>519,1337362,18488,2909


In [28]:
f2_all.to_csv("../interm/f2_all.csv.gz", index=False, compression="gzip")

In [29]:
f2_all[["events:document_id","ad_prom:document_id"]].head(10)

,events:document_id,ad_prom:document_id
0,379743,938164
1,379743,938164
2,379743,1085937
3,379743,1085937
4,379743,1337362
5,379743,1337362
6,379743,992370
7,379743,992370
8,379743,1670176
9,379743,1670176


## join document meta and document aux data
- join with document meta